In [ ]:
itt = ift.train_set_from_dir('data/hadoop_images/test')

In [ ]:
test = ift.train_set_from_dir('data/hadoop_images/test')

In [ ]:
train = ift.train_set_from_dir('data/hadoop_images/train/')

In [ ]:
PIL.Image.open('data/0_200x200.png')

## PIL.Image.open('data/1_200x200.png')

In [ ]:
with open('data/hadoop/test.java', 'rb') as test_java, open('data/hadoop/train.java', 'rb') as train_java:
    test_text = test_java.read()
    train_text = train_java.read()
    


In [1]:
import autoencoders.images_from_text as ift
reload(ift)
import os
import PIL
import PIL.Image
import PIL.ImageFont
import PIL.ImageOps
import PIL.ImageDraw
from keras.preprocessing.image import img_to_array, array_to_img

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN not available)


In [3]:

from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Convolution2D, Deconvolution2D, MaxPooling2D
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras.datasets import mnist

# input image dimensions
img_rows, img_cols, img_chns = 200, 200, 1
# number of convolutional filters to use
nb_filters = 32
# convolution kernel size
nb_conv = 3

batch_size = 16
original_dim = (img_chns, img_rows, img_cols)
latent_dim = 2
intermediate_dim = 128
epsilon_std = 0.01
nb_epoch = 5


x = Input(batch_shape=(batch_size,) + original_dim)
c = Convolution2D(nb_filters, nb_conv, nb_conv, border_mode='same', activation='relu')(x)
f = Flatten()(c)
h = Dense(intermediate_dim, activation='relu')(f)

z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)


def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim),
                              mean=0., std=epsilon_std)
    return z_mean + K.exp(z_log_var) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
# so you could write `Lambda(sampling)([z_mean, z_log_var])`
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_f = Dense(nb_filters*img_rows*img_cols, activation='relu')
decoder_c = Reshape((nb_filters, img_rows, img_cols))
decoder_mean = Deconvolution2D(img_chns, nb_conv, nb_conv,
                               (batch_size, img_chns, img_rows, img_cols),
                               border_mode='same')

h_decoded = decoder_h(z)
f_decoded = decoder_f(h_decoded)
c_decoded = decoder_c(f_decoded)
x_decoded_mean = decoder_mean(c_decoded)


def vae_loss(x, x_decoded_mean):
    # NOTE: binary_crossentropy expects a batch_size by dim for x and x_decoded_mean, so we MUST flatten these!
    x = K.flatten(x)
    x_decoded_mean = K.flatten(x_decoded_mean)
    xent_loss = objectives.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss

vae = Model(x, x_decoded_mean)
vae.compile(optimizer='rmsprop', loss=vae_loss)
vae.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (16, 1, 200, 200)     0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (16, 32, 200, 200)    320         input_1[0][0]                    
____________________________________________________________________________________________________
flatten_1 (Flatten)              (16, 1280000)         0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
dense_1 (Dense)                  (16, 128)             163840128   flatten_1[0][0]                  
___________________________________________________________________________________________

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32')[:, None, :, :] / 255.
x_test = x_test.astype('float32')[:, None, :, :] / 255.

x_train = x_train[:6000, :]
x_test = x_test[:2000, :]
x_test.shape, x_train.shape


((2000, 1, 28, 28), (6000, 1, 28, 28))

In [ ]:
# x_train = test[:20000, :].astype('float32') / 255.
# x_test = test[-4600:, :].astype('float32') / 255.
x_train = test[:20000, :].astype('float32')[:, None, :, :] / 255.
x_test = test[-4600:, :].astype('float32')[:, None, :, :] / 255.
x_train.shape, x_test.shape

In [ ]:
vae.fit(x_train, x_train,
        shuffle=True,
        nb_epoch=1,
        batch_size=batch_size,
        validation_data=(x_test, x_test))

In [ ]:
# build a model to project inputs on the latent space
encoder = Model(x, z_mean)


decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
new_img_generator = Model(decoder_input, _x_decoded_mean)



In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:

from IPython.display import display
for a in range(-200, 200, 100):
    for b in range(-200, 200, 100):
        ara = new_img_generator.predict(np.array([[a,b]]))
        ima = array_to_img(ara.reshape((1, img_width, img_height))).resize((250, 250))
        print a, b
        display(ima)

In [ ]:
x_test.shape

In [ ]:
x_test = 